In [51]:
import os
import geopandas as gpd
import rasterio
from rasterio.transform import from_origin

def tif_shp2df_voc_pascal(image_path, shapefile_path, area_remove = 20):
    # Đọc tệp tin Shapefile vào GeoDataFrame
    gdf = gpd.read_file(shapefile_path)
    gdf_met = gdf.to_crs(epsg=3857)
    gdf = gdf[gdf_met.area > area_remove]
    
    # Mở ảnh TIFF với rasterio
    with rasterio.open(image_path) as src:
        # Lấy thông tin về biến đổi tọa độ
        transform = src.transform
        gdf['filename'] = os.path.basename(image_path).replace('.tif', '.jpg')
        gdf['width'] = src.width
        gdf['height'] = src.height
        gdf['class'] = 'ship'
        # Chuyển đổi từ tọa độ không gian sang tọa độ pixel
        gdf['xmin'] = ((gdf.geometry.bounds['minx'] - transform[2]) / transform[0]).astype(int)
        gdf['ymin'] = ((gdf.geometry.bounds['maxy'] - transform[5]) / transform[4]).astype(int)
        gdf['xmax'] = ((gdf.geometry.bounds['maxx'] - transform[2]) / transform[0]).astype(int)
        gdf['ymax'] = ((gdf.geometry.bounds['miny'] - transform[5]) / transform[4]).astype(int)

    # Hiển thị GeoDataFrame đã chuyển đổi
    gdf = gdf.drop(columns=['FID', 'geometry'])
    # print(gdf)
    return gdf

# image_path = r'/home/skm/SKM16/IMAGE/ZZ_ZZ/TauBien/data_faster-rnn/image_crop_box/tmp/gen_TauBien_Original_3band_cut_256_stride_128_V2/images_data/val/images/DaNang_23_07_2009_LowAccuracy_4_3.tif'
# shapefile_path = r'/home/skm/SKM16/IMAGE/ZZ_ZZ/TauBien/data_faster-rnn/image_crop_box/tmp/gen_TauBien_Original_3band_cut_256_stride_128_V2/images_data/val/images_shape/DaNang_23_07_2009_LowAccuracy_4_3.shp'
# tif_shp2df_voc_pascal(image_path, shapefile_path)

## Chuyen tif sang jpg

In [52]:
import os
from PIL import Image
def tif2jpg(fp_tif, fp_jpg):
    # Đọc ảnh từ định dạng TIFF
    image = Image.open(fp_tif)

    # Chuyển đổi và lưu ảnh dưới định dạng JPEG
    image.save(fp_jpg)
    print(f'Done {os.path.basename(fp_jpg)}')

## Run with all folder

In [57]:
import glob, os
import pandas as pd

list_folder = ['train', 'val']
for name_folder in list_folder:
    dir_img_tif = f'/home/skm/SKM16/IMAGE/ZZ_ZZ/TauBien/data_faster-rnn/image_crop_box/tmp/gen_TauBien_Original_3band_cut_256_stride_128_V2/images_data/{name_folder}/images'
    dir_shp = f'/home/skm/SKM16/IMAGE/ZZ_ZZ/TauBien/data_faster-rnn/image_crop_box/tmp/gen_TauBien_Original_3band_cut_256_stride_128_V2/images_data/{name_folder}/images_shape'
    dir_out_format_csv_and_jpg = r'/home/skm/SKM16/IMAGE/ZZ_ZZ/TauBien/code/Try/SSD/maskrcnn/data_256'
    fp_csv = os.path.join(dir_out_format_csv_and_jpg, f'{name_folder}.csv')
    
    # make folder jpg
    print(name_folder)
    dir_store_jpg = os.path.join(dir_out_format_csv_and_jpg, name_folder)
    os.makedirs(dir_store_jpg, exist_ok=True)

    list_of_dataframes = []
    # tat ca anh
    list_fp_tif = glob.glob(os.path.join(dir_img_tif, '*.tif'))
    for fp_tif in list_fp_tif:
        fp_shp = os.path.join(dir_shp, os.path.basename(fp_tif).replace('.tif', '.shp'))
        fp_jpg = os.path.join(dir_store_jpg, os.path.basename(fp_tif).replace('.tif', '.jpg'))
        df = tif_shp2df_voc_pascal(fp_tif, fp_shp)
        list_of_dataframes.append(df)
        tif2jpg(fp_tif, fp_jpg)
    # Ghép các DataFrame trong danh sách thành một DataFrame duy nhất
    merged_df = pd.concat(list_of_dataframes, ignore_index=True)
    merged_df.to_csv(fp_csv, index=False)
    print('done')


train
Done DaNang_23_07_2009_LowAccuracy_4_80.jpg
Done DaNang_23_07_2009_LowAccuracy_0_2.jpg
Done DaNang_23_07_2009_LowAccuracy_0_6.jpg
Done DaNang_23_07_2009_LowAccuracy_0_7.jpg
Done DaNang_23_07_2009_LowAccuracy_1_1033.jpg
Done DaNang_23_07_2009_LowAccuracy_1_1035.jpg
Done DaNang_23_07_2009_LowAccuracy_1_1043.jpg
Done DaNang_23_07_2009_LowAccuracy_1_1045.jpg
Done DaNang_23_07_2009_LowAccuracy_1_1046.jpg
Done DaNang_23_07_2009_LowAccuracy_1_1048.jpg
Done DaNang_23_07_2009_LowAccuracy_1_1049.jpg
Done DaNang_23_07_2009_LowAccuracy_1_1105.jpg
Done DaNang_23_07_2009_LowAccuracy_1_1106.jpg
Done DaNang_23_07_2009_LowAccuracy_1_1108.jpg
Done DaNang_23_07_2009_LowAccuracy_1_1109.jpg
Done DaNang_23_07_2009_LowAccuracy_1_1110.jpg
Done DaNang_23_07_2009_LowAccuracy_1_1112.jpg
Done DaNang_23_07_2009_LowAccuracy_4_87.jpg
Done DaNang_23_07_2009_LowAccuracy_4_89.jpg
Done DaNang_23_07_2009_LowAccuracy_4_97.jpg
Done DaNang_23_07_2009_LowAccuracy_4_98.jpg
Done DaNang_23_07_2009_LowAccuracy_5_1.jpg
Done

## DEN DAY SU DUNG ENV: MYAI de gen tf_record

In [ ]:
# 8. CSV to TFRecord. With help from MiAI git hub
%cd /home/skm/SKM16/IMAGE/ZZ_ZZ/TauBien/code/MiAI_SSD_TF2/models/research
# !git clone https://github.com/thangnch/MiAI_SSD_TF2
# !cp MiAI_SSD_TF2/generate_tfrecord.py .
# !cp MiAI_SSD_TF2/label_map.txt /home/skm/SKM16/IMAGE/ZZ_ZZ/TauBien/code/MiAI_SSD_TF2/data

import os
if not os.path.exists("/home/skm/SKM16/IMAGE/ZZ_ZZ/TauBien/code/Try/SSD/maskrcnn/data_256/tf_records/"):
  os.mkdir("/home/skm/SKM16/IMAGE/ZZ_ZZ/TauBien/code/Try/SSD/maskrcnn/data_256/tf_records/")

!python generate_tfrecord.py --image_dir=/home/skm/SKM16/IMAGE/ZZ_ZZ/TauBien/code/Try/SSD/maskrcnn/data_256/train --csv_input=/home/skm/SKM16/IMAGE/ZZ_ZZ/TauBien/code/Try/SSD/maskrcnn/data_256/train_labels.csv --output_path=/home/skm/SKM16/IMAGE/ZZ_ZZ/TauBien/code/Try/SSD/maskrcnn/data_256/tf_records/train.record 
!python generate_tfrecord.py --image_dir=/home/skm/SKM16/IMAGE/ZZ_ZZ/TauBien/code/Try/SSD/maskrcnn/data_256/test --csv_input=/home/skm/SKM16/IMAGE/ZZ_ZZ/TauBien/code/Try/SSD/maskrcnn/data_256/test_labels.csv --output_path=/home/skm/SKM16/IMAGE/ZZ_ZZ/TauBien/code/Try/SSD/maskrcnn/data_256/tf_records/test.record 